In [2]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly_express as px


## 1.1

Importación de archivos

In [3]:

data_train = pd.read_csv("https://practicum-content.s3.us-west-1.amazonaws.com/datasets/gold_recovery_train.csv")
data_test = pd.read_csv("https://practicum-content.s3.us-west-1.amazonaws.com/datasets/gold_recovery_test.csv")
data_full = pd.read_csv("https://practicum-content.s3.us-west-1.amazonaws.com/datasets/gold_recovery_full.csv")

## 1.2

Comprueba que el calculo de recuperación sea correcto

In [4]:
#Columna con calculo de recuperación: data_train["rougher.output.recovery"]

# Función que calcula el % de recuperación
def recovery(df):
    c = df["rougher.output.concentrate_au"]
    f = df["rougher.input.feed_au"]
    t = df["rougher.output.tail_au"]
    try:
        result = ((c*(f-t))/(f*(c-t)))*100
    except:
        result = 0
    return result

recovery_calc = data_train.apply(lambda x: recovery(x), axis = 1)

print("Error absoluto medio del cálculo de recuperación:", (recovery_calc - data_train["rougher.output.recovery"]).abs().mean())

Error absoluto medio del cálculo de recuperación: 9.303415616264301e-15


Al tener un error absoluto medio tan bajo, podemos afirmar que los calculos han sido correctos hasta el momento

## 1.3

Analiza las características no disponibles del conjunto de prueba

In [5]:
not_in_test = set(data_train.columns) - set(data_test.columns)
not_in_test

{'final.output.concentrate_ag',
 'final.output.concentrate_au',
 'final.output.concentrate_pb',
 'final.output.concentrate_sol',
 'final.output.recovery',
 'final.output.tail_ag',
 'final.output.tail_au',
 'final.output.tail_pb',
 'final.output.tail_sol',
 'primary_cleaner.output.concentrate_ag',
 'primary_cleaner.output.concentrate_au',
 'primary_cleaner.output.concentrate_pb',
 'primary_cleaner.output.concentrate_sol',
 'primary_cleaner.output.tail_ag',
 'primary_cleaner.output.tail_au',
 'primary_cleaner.output.tail_pb',
 'primary_cleaner.output.tail_sol',
 'rougher.calculation.au_pb_ratio',
 'rougher.calculation.floatbank10_sulfate_to_au_feed',
 'rougher.calculation.floatbank11_sulfate_to_au_feed',
 'rougher.calculation.sulfate_to_au_concentrate',
 'rougher.output.concentrate_ag',
 'rougher.output.concentrate_au',
 'rougher.output.concentrate_pb',
 'rougher.output.concentrate_sol',
 'rougher.output.recovery',
 'rougher.output.tail_ag',
 'rougher.output.tail_au',
 'rougher.output.ta

Podemos observar que las columnas que no se encuentran en el conjunto de prueba son columnas tipo "output", valores que se obtienes después del procesado del material, es decir, valores futuros.

## 1.4

Preprocesamiento de datos

Para poder hacer un preprocesamiento adecuado, debemos tomar en cuenta 2 puntos importantes, el primero es que los datos cercanos en el tiempo suelen ser similares, y el segundo es que contamos con parámetros de fecha y hora  para poder definir datos cercanos.

 Buscamos la desviación estandar de 3 muestras de cada columna para corroborar que los datos realmente se acercan entre sí en fechas cercanas, los datos en los df están ordenados por fecha y hora de registro

In [6]:
for _ in range(3):
    rd = np.random.randint(0,22000)
    print(data_full.loc[rd:rd+5, :].drop(["date"], axis=1).std())

final.output.concentrate_ag                   0.000000
final.output.concentrate_pb                   0.000000
final.output.concentrate_sol                  0.000000
final.output.concentrate_au                   0.000000
final.output.recovery                              NaN
                                                ...   
secondary_cleaner.state.floatbank5_a_level    0.093058
secondary_cleaner.state.floatbank5_b_air      0.533419
secondary_cleaner.state.floatbank5_b_level    0.044442
secondary_cleaner.state.floatbank6_a_air      1.056307
secondary_cleaner.state.floatbank6_a_level    0.497364
Length: 86, dtype: float64
final.output.concentrate_ag                     1.878131
final.output.concentrate_pb                     3.541706
final.output.concentrate_sol                    1.915552
final.output.concentrate_au                    15.629387
final.output.recovery                          14.891609
                                                 ...    
secondary_cleaner.state.fl

Al tener una desviación estandar normalmente no tan alejada en las columnas, sustituiremos los valores vacíos con los valores de el dato anterior más proximo en fecha

In [7]:
for data in [data_full,data_test,data_train]:
    data.ffill(inplace=True)
    data.isnull().sum()

## 2.1

Observa cómo cambia la concentracion de metales en función de cada etapa

In [8]:
columnas = pd.Series(data_full.columns)
au1 = data_full["rougher.input.feed_au"].mean()
au3 = data_full["rougher.output.concentrate_au"].mean()
au5 = data_full["primary_cleaner.output.concentrate_au"].mean()
au8 = data_full["final.output.concentrate_au"].mean()
oro = [au1, au3, au5, au8]

ag1 = data_full["rougher.input.feed_ag"].mean()
ag3 = data_full["rougher.output.concentrate_ag"].mean()
ag5 = data_full["primary_cleaner.output.concentrate_ag"].mean()
ag8 = data_full["final.output.concentrate_ag"].mean()
plata = [ag1, ag3, ag5, ag8]

pb1 = data_full["rougher.input.feed_pb"].mean()
pb3 = data_full["rougher.output.concentrate_pb"].mean()
pb5 = data_full["primary_cleaner.output.concentrate_pb"].mean()
pb8 = data_full["final.output.concentrate_pb"].mean()
plomo = [pb1, pb3, pb5, pb8]

stages = ['rougher_input', 'rougher_output', 'primary_cleaner', 'final_output']

fig = go.Figure(data=[
    go.Bar(name='Oro', x=stages, y=oro ),
    go.Bar(name='Plata', x=stages, y=plata),
    go.Bar(name='Plomo', x=stages, y=plomo)
])
# Change the bar mode
fig.update_layout(barmode='group', title_text = "Composición de la mezcla de metales en diferentes etapas")
fig.show()

Podemos ver un incremento constante de la cantidad de oro en la mezcla cuando las etapas son más avanzadas, mientras que la plata decrece despues de la salida rougher y el plomo llega a un punto máximo en primary cleaner

## 2.1



## 2.1



## 2.1



## 2.1

